# Word2Vec: Understanding Dense Word Embeddings from the Ground Up

## Introduction: Why Dense Embeddings Matter

Imagine you're trying to teach a computer to understand language. You could represent each word as a unique ID number (like word #1234 = "cat"), but this tells us nothing about what words mean or how they relate to each other. This is the fundamental limitation of sparse representations that we've previously discussed.

Dense embeddings revolutionize this by representing each word as a point in a continuous vector space - think of it as placing words in a multi-dimensional universe where similar words are close together and dissimilar words are far apart. Today, we'll dive deep into Word2Vec, one of the most influential algorithms for creating these dense embeddings.

### 🧠 Learning Objective
By the end of this notebook, you will:
1. Understand the mathematical foundations of Word2Vec
2. Master both the Skip-gram and CBOW architectures 
3. Implement key components of Word2Vec from scratch
4. Appreciate why Word2Vec was revolutionary for NLP

---

## Part 1: The Intuition Behind Word2Vec

### From Sparse to Dense: A Paradigm Shift

Let's start with a crucial insight: **words that appear in similar contexts tend to have similar meanings**. This is known as the distributional hypothesis, famously summarized by linguist J.R. Firth: "You shall know a word by the company it keeps."


### The Key Innovation: Predicting Context from Words

The Key Innovation: From Counting to Predicting
This is a crucial concept, as it represents one of the most important paradigm shifts in NLP.

### Understanding the Revolutionary Shift
The Old Way: Counting Co-occurrences
Before Word2Vec, most word representation methods relied on counting. Let me illustrate this with a concrete example:





In [4]:
import numpy as np

# Traditional co-occurrence counting approach
corpus = [
    "I love machine learning",
    "I love deep learning", 
    "Machine learning is powerful",
    "Deep learning requires data"
]

# Make vocab all lowercase to match your .lower()’ed words:
vocab = ["i", "love", "machine", "learning", "deep", "is", "powerful", "requires", "data"]

cooccurrence_matrix = np.zeros((len(vocab), len(vocab)), dtype=int)

window_size = 1
for sentence in corpus:
    words = sentence.lower().split()
    for i, word1 in enumerate(words):
        for j in range(max(0, i-window_size), min(len(words), i+window_size+1)):
            if i == j:
                continue
            idx1 = vocab.index(word1)
            idx2 = vocab.index(words[j])
            cooccurrence_matrix[idx1, idx2] += 1

print("Co-occurrence matrix (partial):")
print(cooccurrence_matrix[:5, :5])

Co-occurrence matrix (partial):
[[0 2 0 0 0]
 [2 0 1 0 1]
 [0 1 0 2 0]
 [0 0 2 0 2]
 [0 1 0 2 0]]


## Limitations of the Traditional Approach

This approach has several limitations:

- **Sparsity**: Most word pairs never co-occur, leading to mostly zero-filled matrices  
- **High dimensionality**: Each word needs a dimension for every other word  
- **Poor generalization**: Can't infer relationships between words that don't directly co-occur  
- **Memory intensive**: Storing these large, sparse matrices is inefficient  

---

## The New Way: Prediction as Learning

Word2Vec's genius was to reformulate the problem.  
Instead of asking **"How often do words appear together?"**, it asks:  
**"Can I predict which words will appear together?"**

Let me break this down with a deeper analogy:

### 🧠 Enhanced Analogy

Imagine you're learning a new language by living in a foreign country. You have two approaches:

- **Approach 1 (Traditional)**:  
  You carry a notebook and tally every time you hear word pairs together.  
  "Coffee" appears with "hot" 23 times, with "cup" 18 times, etc.  
  This gives you statistics but not understanding.

- **Approach 2 (Word2Vec)**:  
  You try to predict what words you'll hear next.  
  When someone says *"I'd like a cup of..."*, you predict "coffee" or "tea".  
  When you're wrong, you adjust your mental model.  
  Over time, you develop an intuition that **"coffee"**, **"tea"**, and **"water"** are similar because they appear in similar contexts.

---

This **prediction-based approach** naturally captures **semantic relationships**,  
because **words with similar meanings tend to appear in similar contexts**.


Word2Vec's breakthrough was treating word embedding as a **prediction problem**. Instead of counting co-occurrences (like in sparse methods), Word2Vec:

1. Takes a word as input
2. Tries to predict its surrounding context words
3. Adjusts word vectors to improve these predictions
4. The resulting vectors capture semantic relationships!



---

## 📖 Understanding Word2Vec: The Complete Picture

### 🎯 Now that we have an intuition for how Word2Vec differs from traditional sparse representations, let's give it a **clear, formal definition**.

---

### ❓ What Exactly is Word2Vec?

**Word2Vec** is not a single algorithm, but rather a **family of model architectures** designed to learn **dense vector representations of words**, known as **word embeddings**.

Think of Word2Vec as a **framework** for capturing semantic and syntactic relationships between words by leveraging the **distributional hypothesis**:  
> *Words that occur in similar contexts tend to have similar meanings.*

Introduced by **Tomas Mikolov et al.** at Google in **2013**, Word2Vec transforms each word in a corpus into a **low-dimensional, dense vector** that captures its contextual usage.


### Understanding Word2Vec: The Complete Picture

#### What Exactly is Word2Vec?

Word2Vec is **not** a single algorithm—rather, it is a *family of model architectures* for learning dense, numerical representations of words (known as *word embeddings*). Think of Word2Vec as a toolkit that offers **two complementary ways** to learn these vectors:

| Model Architecture | Core Idea | Direction of Prediction |
|--------------------|-----------|-------------------------|
| **Skip‑gram** | Uses one *target* word to predict the words that appear around it. | **Target → Context** |
| **Continuous Bag of Words (CBOW)** | Uses the surrounding *context* words to predict the missing target word. | **Context → Target** |

When people say **“Word2Vec,”** they’re talking about this overall framework, first introduced by *Mikolov et al.* at Google in 2013. The name literally means **“Word to Vector”**—turning words into vectors that a machine‑learning model can manipulate.

---

### What do we mean by “model architecture”?

In machine‑learning jargon, a **model architecture** is the **high‑level blueprint** that specifies:

- **Input & output format** – what goes in, what comes out (e.g., a word vs. its context).
- **Computation flow** – the sequence of layers or operations that transform inputs into outputs.
- **Objective function** – the metric the model optimizes (e.g., predicting surrounding words).

In **Word2Vec**, **Skip‑gram** and **CBOW** are two different architectures because they have opposite prediction goals and therefore different data flows, even though they share the same overall purpose (learning word embeddings). This distinction matters because it affects training speed, data efficiency, and the kinds of semantic relationships that emerge in the learned vectors.

Let's illustrate this relationship:

In [ ]:
class Word2VecFramework:
    """
    Word2Vec is the overarching framework containing both Skip-gram and CBOW
    """
    def __init__(self, architecture="skip-gram"):
        self.architecture = architecture
        print(f"Word2Vec Framework initialized with {architecture} architecture")
    
    def explain_relationship(self):
        explanation = """
        Word2Vec Framework
        ├── Skip-gram Model
        │   └── Predicts context from target
        │       Example: "fox" → ["quick", "brown", "jumps", "over"]
        │
        └── CBOW Model
            └── Predicts target from context
                Example: ["quick", "brown", "jumps", "over"] → "fox"
        
        Both models share:
        - The goal of learning dense word embeddings
        - Similar optimization techniques (negative sampling, hierarchical softmax)
        - The same underlying principle: distributional semantics


        
        """
        print(explanation)

# Demonstrate the relationship
word2vec = Word2VecFramework()
word2vec.explain_relationship()

Word2Vec Framework initialized with skip-gram architecture

        Word2Vec Framework
        ├── Skip-gram Model
        │   └── Predicts context from target
        │       Example: "fox" → ["quick", "brown", "jumps", "over"]
        │
        └── CBOW Model
            └── Predicts target from context
                Example: ["quick", "brown", "jumps", "over"] → "fox"
        
        Both models share:
        - The goal of learning dense word embeddings
        - Similar optimization techniques (negative sampling, hierarchical softmax)
        - The same underlying principle: distributional semantics
        



to sum up.
Word2Vec comes in two core flavors:

- **Skip-gram**:  
  Given a **target word**, the model tries to predict its surrounding **context words**.  
  This architecture works well with **large, sparse datasets** and is particularly effective at capturing **rare word semantics**.

- **Continuous Bag of Words (CBOW)**:  
  Given a set of **context words**, the model tries to predict the **target word**.  
  This approach is typically faster and works better with **frequent words**.

These two approaches share the same goal:
➡️ Learn **word vectors** such that words with **similar contexts** are mapped to **similar embeddings** in vector space.

---



## ⚙️ How Does Word2Vec Work?

1. **Training Objective**  
   Word2Vec treats word relationships as a **prediction problem**, not a counting problem.  
   Using a shallow neural network (typically 1 hidden layer), the model is trained to **maximize the probability** of observed word-context pairs.

2. **Input and Output**  
   - The **input layer** takes one-hot encoded vectors representing words.  
   - The **hidden layer** contains the **learned word embeddings**.  
   - The **output layer** predicts probabilities over the vocabulary using a softmax function (or approximations like negative sampling).

3. **Optimization**  
   Word2Vec uses techniques like:
   - **Negative Sampling**: Instead of updating the full softmax over all words, it updates only a small subset of "negative" samples.
   - **Hierarchical Softmax**: Speeds up training by organizing the vocabulary in a binary tree.

---

### 🔍 Skip-gram in Action

Let’s say your corpus contains the sentence:

> "The cat sat on the mat."

If the target word is `"sat"` and the context window size is 2,  
the Skip-gram model learns to predict:

- Input: `"sat"`  
- Output: `"cat"`, `"on"`

---

### 💡 Why Does This Work?

Over time, the network adjusts the word vectors so that words occurring in **similar contexts** end up having **similar vector representations**.  
This results in a **semantic space** where analogies like:

> *king - man + woman ≈ queen*

are **arithmetically meaningful**.

---

### 🌍💵 Real-world Impact

Word2Vec revolutionized NLP by making it possible to:

- Cluster semantically similar words
- Improve text classification and search
- Enable analogical reasoning in vector space
- Pretrain embeddings for deep learning models

---




---


## Part 2: The Skip-gram Architecture

### Core Concept: Predicting Context from Target

The skip-gram model takes a target word and predicts surrounding context words. Let's break this down step by step.

Given the sentence: "The quick brown fox jumps over the lazy dog"

If our target word is "fox" and we use a context window of size 2, we want to predict:
- Context words: ["quick", "brown", "jumps", "over"]

### Mathematical Foundation

The skip-gram model uses two embedding matrices:
- **W** ∈ ℝ^(|V|×d): Target word embeddings (input embeddings)
- **C** ∈ ℝ^(|V|×d): Context word embeddings (output embeddings)

Where:
- |V| is vocabulary size
- d is embedding dimension

For a target word w and context word c, the probability is:

P(c|w) = exp(c·w) / Σ(exp(c'·w)) for all c' in V

> ⚠️ **Critical Insight**: We maintain TWO sets of embeddings! This separation helps training stability and often produces better results.


In [2]:
import numpy as np

class SkipGramDemo:
    def __init__(self, vocab_size, embedding_dim):
        # Initialize two embedding matrices
        self.W = np.random.randn(vocab_size, embedding_dim) * 0.01  # Target embeddings
        self.C = np.random.randn(vocab_size, embedding_dim) * 0.01  # Context embeddings
        
    def compute_probability(self, target_idx, context_idx):
        """
        Compute P(context|target) using softmax
        
        Args:
            target_idx: Index of target word
            context_idx: Index of context word
        
        Returns:
            probability: P(context|target)
        """
        # Get embeddings
        target_embedding = self.W[target_idx]
        context_embedding = self.C[context_idx]
        
        # Compute dot product
        score = np.dot(target_embedding, context_embedding)
        
        # Compute softmax denominator (sum over all context words)
        all_scores = np.dot(self.C, target_embedding)
        exp_scores = np.exp(all_scores - np.max(all_scores))  # Numerical stability
        
        probability = exp_scores[context_idx] / np.sum(exp_scores)
        return probability

# Demonstration
vocab_size, embedding_dim = 10000, 100
model = SkipGramDemo(vocab_size, embedding_dim)

# Example computation
target_word_idx = 42  # Index for "fox"
context_word_idx = 156  # Index for "quick"
prob = model.compute_probability(target_word_idx, context_word_idx)
print(f"P(quick|fox) = {prob:.6f}")

P(quick|fox) = 0.000100




### The Computational Challenge: Softmax

The softmax computation requires summing over the entire vocabulary, making it computationally expensive for large vocabularies. This leads us to...

---

## Part 3: Negative Sampling - Making Word2Vec Practical

### The Problem with Full Softmax

Computing the softmax denominator requires O(|V|) operations per training example. With vocabularies often exceeding 100,000 words, this becomes prohibitively expensive.

### Solution: Negative Sampling

Instead of updating all word vectors for each training example, negative sampling:
1. Updates the target word vector
2. Updates the positive context word vector
3. Updates only k randomly selected "negative" context word vectors

The new objective function becomes:

log σ(c_pos·w) + Σ[k negative samples] log σ(-c_neg·w)

Where σ(x) = 1/(1+exp(-x)) is the sigmoid function.



```python
def sigmoid(x):
    """Numerically stable sigmoid function"""
    return np.where(x >= 0, 
                    1 / (1 + np.exp(-x)), 
                    np.exp(x) / (1 + np.exp(x)))

class SkipGramNegativeSampling:
    def __init__(self, vocab_size, embedding_dim):
        self.W = np.random.randn(vocab_size, embedding_dim) * 0.01
        self.C = np.random.randn(vocab_size, embedding_dim) * 0.01
        self.vocab_size = vocab_size
        
    def train_pair(self, target_idx, context_idx, negative_samples, learning_rate=0.025):
        """
        Train on one target-context pair with negative sampling
        
        Args:
            target_idx: Target word index
            context_idx: Positive context word index
            negative_samples: List of negative sample indices
            learning_rate: Learning rate for SGD
        """
        # Get embeddings
        target_vec = self.W[target_idx]
        context_vec = self.C[context_idx]
        
        # Positive example gradient
        score = np.dot(target_vec, context_vec)
        sigmoid_score = sigmoid(score)
        grad_target = (sigmoid_score - 1) * context_vec
        grad_context = (sigmoid_score - 1) * target_vec
        
        # Update for positive example
        self.W[target_idx] -= learning_rate * grad_target
        self.C[context_idx] -= learning_rate * grad_context
        
        # Negative examples
        for neg_idx in negative_samples:
            neg_vec = self.C[neg_idx]
            score = np.dot(target_vec, neg_vec)
            sigmoid_score = sigmoid(score)
            
            # Gradients for negative examples (note the sign difference)
            grad_target_neg = sigmoid_score * neg_vec
            grad_neg = sigmoid_score * target_vec
            
            # Update
            self.W[target_idx] -= learning_rate * grad_target_neg
            self.C[neg_idx] -= learning_rate * grad_neg

# Example usage
model = SkipGramNegativeSampling(vocab_size=10000, embedding_dim=100)
target_idx = 42
context_idx = 156
negative_samples = np.random.randint(0, 10000, size=5)
model.train_pair(target_idx, context_idx, negative_samples)
```

### Choosing Negative Samples

Word2Vec doesn't sample negatives uniformly. Instead, it uses:

P(w) = count(w)^(3/4) / Σ count(w')^(3/4)

This formula raises word frequencies to the 3/4 power, which:
- Increases probability of selecting rare words
- Decreases probability of selecting very common words
- Results in better embeddings overall

```python
def create_negative_sampling_distribution(word_counts, power=0.75):
    """
    Create distribution for negative sampling
    
    Args:
        word_counts: Dictionary mapping word indices to their counts
        power: Power to raise frequencies (typically 0.75)
    
    Returns:
        sampling_probs: Array of sampling probabilities
    """
    vocab_size = len(word_counts)
    counts = np.array([word_counts.get(i, 0) for i in range(vocab_size)])
    
    # Raise to power and normalize
    powered_counts = counts ** power
    sampling_probs = powered_counts / np.sum(powered_counts)
    
    return sampling_probs

# Example
word_counts = {i: np.random.zipf(1.5) for i in range(1000)}
sampling_dist = create_negative_sampling_distribution(word_counts)

# Visualize the effect
plt.figure(figsize=(10, 6))
original_probs = np.array(list(word_counts.values())) / sum(word_counts.values())
plt.scatter(original_probs, sampling_dist, alpha=0.5)
plt.xlabel('Original Probability')
plt.ylabel('Negative Sampling Probability')
plt.title('Effect of Power Scaling on Sampling Distribution')
plt.xscale('log')
plt.yscale('log')
plt.plot([1e-5, 1e-1], [1e-5, 1e-1], 'r--', label='y=x')
plt.legend()
plt.show()
```

---

## Part 4: Training Word2Vec - The Complete Algorithm

Let's put everything together to understand the full training process:

```python
class Word2Vec:
    def __init__(self, sentences, embedding_dim=100, window_size=5, 
                 negative_samples=5, epochs=5, learning_rate=0.025):
        # Build vocabulary
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.word_counts = {}
        self.build_vocabulary(sentences)
        
        # Initialize embeddings
        self.vocab_size = len(self.word_to_idx)
        self.W = np.random.randn(self.vocab_size, embedding_dim) * 0.01
        self.C = np.random.randn(self.vocab_size, embedding_dim) * 0.01
        
        # Training parameters
        self.window_size = window_size
        self.negative_samples = negative_samples
        self.epochs = epochs
        self.initial_lr = learning_rate
        
        # Create negative sampling distribution
        self.create_sampling_table()
        
    def build_vocabulary(self, sentences):
        """Build vocabulary from sentences"""
        idx = 0
        for sentence in sentences:
            for word in sentence:
                if word not in self.word_to_idx:
                    self.word_to_idx[word] = idx
                    self.idx_to_word[idx] = word
                    self.word_counts[idx] = 0
                    idx += 1
                self.word_counts[self.word_to_idx[word]] += 1
    
    def create_sampling_table(self):
        """Create table for negative sampling"""
        power = 0.75
        norm = sum([count**power for count in self.word_counts.values()])
        
        table_size = 1e8
        table = []
        
        pow_frequency = [count**power / norm for count in self.word_counts.values()]
        word_index = 0
        total_pow_freq = pow_frequency[word_index]
        
        for i in range(int(table_size)):
            table.append(word_index)
            if i / table_size > total_pow_freq:
                word_index += 1
                if word_index < len(self.word_counts):
                    total_pow_freq += pow_frequency[word_index]
                else:
                    word_index = len(self.word_counts) - 1
        
        self.sampling_table = np.array(table)
    
    def get_negative_samples(self, target_idx, context_idx):
        """Get negative samples, avoiding target and context"""
        negative_samples = []
        while len(negative_samples) < self.negative_samples:
            neg_idx = self.sampling_table[np.random.randint(len(self.sampling_table))]
            if neg_idx != target_idx and neg_idx != context_idx:
                negative_samples.append(neg_idx)
        return negative_samples
    
    def train_pair(self, target_idx, context_idx, learning_rate):
        """Train on a single target-context pair"""
        negative_samples = self.get_negative_samples(target_idx, context_idx)
        
        # Forward pass
        target_vec = self.W[target_idx]
        context_vec = self.C[context_idx]
        
        # Positive example
        score = np.dot(target_vec, context_vec)
        sigmoid_score = sigmoid(score)
        
        # Gradients
        grad_target = (sigmoid_score - 1) * context_vec
        grad_context = (sigmoid_score - 1) * target_vec
        
        # Negative examples
        for neg_idx in negative_samples:
            neg_vec = self.C[neg_idx]
            neg_score = np.dot(target_vec, neg_vec)
            neg_sigmoid = sigmoid(neg_score)
            
            grad_target += neg_sigmoid * neg_vec
            self.C[neg_idx] -= learning_rate * neg_sigmoid * target_vec
        
        # Update embeddings
        self.W[target_idx] -= learning_rate * grad_target
        self.C[context_idx] -= learning_rate * grad_context
    
    def train(self, sentences):
        """Train Word2Vec model"""
        for epoch in range(self.epochs):
            total_loss = 0
            word_count = 0
            
            # Linearly decay learning rate
            lr = self.initial_lr * (1 - epoch / self.epochs)
            
            for sentence in sentences:
                sentence_indices = [self.word_to_idx[word] for word in sentence]
                
                for center_pos, center_idx in enumerate(sentence_indices):
                    # Get context window
                    context_start = max(0, center_pos - self.window_size)
                    context_end = min(len(sentence_indices), center_pos + self.window_size + 1)
                    
                    for context_pos in range(context_start, context_end):
                        if context_pos != center_pos:
                            context_idx = sentence_indices[context_pos]
                            self.train_pair(center_idx, context_idx, lr)
                            word_count += 1
            
            print(f"Epoch {epoch+1}/{self.epochs} completed")
    
    def get_embedding(self, word):
        """Get embedding for a word"""
        if word in self.word_to_idx:
            idx = self.word_to_idx[word]
            return self.W[idx]  # Usually just use target embeddings
        else:
            return None
    
    def most_similar(self, word, top_k=5):
        """Find most similar words"""
        if word not in self.word_to_idx:
            return []
        
        word_vec = self.get_embedding(word)
        similarities = np.dot(self.W, word_vec)
        most_similar_indices = np.argsort(similarities)[::-1][1:top_k+1]  # Exclude the word itself
        
        return [(self.idx_to_word[idx], similarities[idx]) for idx in most_similar_indices]

# Example usage with sample data
sample_sentences = [
    ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"],
    ["the", "dog", "barks", "at", "the", "cat"],
    ["the", "cat", "meows", "at", "the", "dog"]
]

model = Word2Vec(sample_sentences, embedding_dim=50, window_size=2, epochs=5)
model.train(sample_sentences)

# Find similar words
similar_words = model.most_similar("dog", top_k=3)
print("Words similar to 'dog':", similar_words)
```

---

## Part 5: Understanding What Word2Vec Learns

### Semantic Relationships

Word2Vec famously captures semantic relationships through vector arithmetic:

```python
def demonstrate_word_arithmetic(model, equation):
    """
    Demonstrate word arithmetic like "king - man + woman = queen"
    
    Args:
        model: Trained Word2Vec model
        equation: String like "king - man + woman"
    """
    words = equation.split()
    result_vec = np.zeros_like(model.W[0])
    
    for i, word in enumerate(words):
        if word in ['+', '-', '=']:
            continue
        
        vec = model.get_embedding(word)
        if vec is None:
            print(f"Word '{word}' not in vocabulary")
            return
        
        if i > 0 and words[i-1] == '-':
            result_vec -= vec
        else:
            result_vec += vec
    
    # Find closest word to result vector
    similarities = np.dot(model.W, result_vec)
    most_similar_idx = np.argmax(similarities)
    
    return model.idx_to_word[most_similar_idx]

# Example (with a properly trained model)
# result = demonstrate_word_arithmetic(model, "king - man + woman")
# print(f"king - man + woman = {result}")  # Should output something close to "queen"
```

### Visualizing Embeddings

We can use dimensionality reduction to visualize high-dimensional embeddings:

```python
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_embeddings(model, words_to_plot=None, perplexity=30):
    """
    Visualize word embeddings using t-SNE
    
    Args:
        model: Trained Word2Vec model
        words_to_plot: List of words to visualize (None = all words)
        perplexity: t-SNE perplexity parameter
    """
    if words_to_plot is None:
        words_to_plot = list(model.word_to_idx.keys())
    
    # Get embeddings for selected words
    embeddings = []
    words = []
    for word in words_to_plot:
        if word in model.word_to_idx:
            embeddings.append(model.get_embedding(word))
            words.append(word)
    
    embeddings = np.array(embeddings)
    
    # Reduce to 2D using t-SNE
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)
    
    # Plot
    plt.figure(figsize=(12, 8))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.6)
    
    for i, word in enumerate(words):
        plt.annotate(word, (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                    xytext=(5, 2), textcoords='offset points')
    
    plt.title('Word Embeddings Visualization (t-SNE)')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.grid(True, alpha=0.3)
    plt.show()

# Example visualization
# visualize_embeddings(model)
```

---

## Part 6: Advanced Concepts and Optimizations

### Subsampling Frequent Words

Very frequent words like "the", "a", "is" provide less information value. Word2Vec uses subsampling to address this:

```python
def subsample_probability(word_freq, threshold=1e-3):
    """
    Calculate probability of keeping a word during training
    
    Args:
        word_freq: Frequency of the word in corpus
        threshold: Subsampling threshold (typically 1e-3 to 1e-5)
    
    Returns:
        probability: Probability of keeping the word
    """
    if word_freq <= 0:
        return 0
    
    prob = 1 - np.sqrt(threshold / word_freq)
    return max(0, prob)

# Visualization of subsampling effect
frequencies = np.logspace(-6, -1, 100)
keep_probs = [1 - subsample_probability(f) for f in frequencies]

plt.figure(figsize=(10, 6))
plt.semilogx(frequencies, keep_probs)
plt.xlabel('Word Frequency')
plt.ylabel('Probability of Keeping Word')
plt.title('Effect of Subsampling on Different Word Frequencies')
plt.grid(True, alpha=0.3)
plt.show()
```

### Hierarchical Softmax (Alternative to Negative Sampling)

Hierarchical softmax organizes words in a binary tree, reducing complexity from O(|V|) to O(log|V|):

```python
class HierarchicalSoftmaxNode:
    def __init__(self, left=None, right=None, word_idx=None, vector=None):
        self.left = left
        self.right = right
        self.word_idx = word_idx
        self.vector = vector if vector is not None else np.random.randn(embedding_dim) * 0.01

def build_huffman_tree(word_freqs):
    """
    Build Huffman tree for hierarchical softmax
    
    Args:
        word_freqs: Dictionary of word frequencies
    
    Returns:
        root: Root node of Huffman tree
    """
    import heapq
    
    # Create leaf nodes
    heap = []
    for word_idx, freq in word_freqs.items():
        node = HierarchicalSoftmaxNode(word_idx=word_idx)
        heapq.heappush(heap, (freq, id(node), node))
    
    # Build tree
    while len(heap) > 1:
        freq1, _, node1 = heapq.heappop(heap)
        freq2, _, node2 = heapq.heappop(heap)
        
        parent = HierarchicalSoftmaxNode(left=node1, right=node2)
        heapq.heappush(heap, (freq1 + freq2, id(parent), parent))
    
    _, _, root = heapq.heappop(heap)
    return root
```

---

## Part 7: Quiz Time! 🧠

Let's test your understanding with some questions:

### Question 1: Core Concepts
What is the fundamental difference between Word2Vec and traditional sparse representations?

<details>
<summary>Click to see answer</summary>

Word2Vec creates dense, continuous vectors that capture semantic relationships, while sparse representations use one-hot or count-based vectors that don't inherently encode semantic similarity. Word2Vec learns these representations by predicting context words, making similar words have similar vectors.
</details>

### Question 2: Architecture
In skip-gram, why do we maintain two separate embedding matrices (W for targets, C for contexts)?

<details>
<summary>Click to see answer</summary>

Having separate matrices:
1. Provides more flexibility in learning
2. Helps avoid trivial solutions where a word predicts itself
3. Often leads to better quality embeddings
4. Allows different roles for words as targets vs. contexts
</details>

### Question 3: Negative Sampling
Why do we raise word frequencies to the power of 3/4 in negative sampling instead of using uniform or raw frequency distributions?

<details>
<summary>Click to see answer</summary>

The 3/4 power:
1. Increases the probability of sampling rare words (compared to their actual frequency)
2. Decreases the probability of sampling very common words
3. Results in a more balanced training signal
4. Leads to better quality embeddings for both common and rare words
</details>

### Question 4: Practical Implementation
What's the time complexity of:
a) Full softmax computation?
b) Negative sampling with k negative samples?

<details>
<summary>Click to see answer</summary>

a) Full softmax: O(|V|) where |V| is vocabulary size
b) Negative sampling: O(k) where k is typically 5-20

This is why negative sampling makes Word2Vec practical for large vocabularies!
</details>

---

## Part 8: Practical Tips for Training Word2Vec

1. **Preprocessing is crucial**:
   - Lowercase text
   - Remove very rare words (appearing < 5 times)
   - Consider replacing numbers with a special token

2. **Hyperparameter guidelines**:
   - Embedding dimension: 100-300 (more dimensions for larger datasets)
   - Window size: 5-10 (smaller for syntactic tasks, larger for semantic tasks)
   - Negative samples: 5-20 (more for larger datasets)
   - Learning rate: Start at 0.025 with linear decay

3. **Training corpus size**:
   - Minimum: 100k words
   - Better results: 1M+ words
   - State-of-the-art: Billions of words

4. **Evaluation during training**:
   - Monitor analogy tasks
   - Check nearest neighbors for common words
   - Use intrinsic evaluation sets like WordSim-353

---

## Conclusion: The Impact of Word2Vec

Word2Vec revolutionized NLP by showing that:
1. Simple prediction tasks can learn rich semantic representations
2. Dense embeddings outperform sparse representations
3. Word meanings can be captured through their contexts
4. Neural approaches can scale to massive datasets

These insights paved the way for modern transformer models and the current revolution in NLP!

### Further Resources 📚

1. **Original Papers**:
   - [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781) (Mikolov et al., 2013)
   - [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546) (Mikolov et al., 2013)

2. **Implementations**:
   - [Gensim Word2Vec Tutorial](https://radimrehurek.com/gensim/models/word2vec.html)
   - [TensorFlow Word2Vec Tutorial](https://www.tensorflow.org/tutorials/text/word2vec)

3. **Advanced Topics**:
   - [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove/)
   - [FastText: Enriching Word Vectors with Subword Information](https://arxiv.org/abs/1607.04606)

4. **Visualization Tools**:
   - [Embedding Projector](https://projector.tensorflow.org/)
   - [Word2Vec Demo](https://ronxin.github.io/wevi/)

Keep practicing and experimenting with these concepts! The journey from Word2Vec to modern language models is fascinating and builds directly on these foundations. 🚀